In [154]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler, OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor, DecisionTreeRegressor, GeneralizedLinearRegression, IsotonicRegression, AFTSurvivalRegression, FMRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import numpy as np


In [155]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [156]:
# Create SparkSession
spark = SparkSession.builder.master('local').appName("HousePricePrediction").getOrCreate()

In [157]:
# Load data from CSV file
data = spark.read.csv("housing.csv", header=True, inferSchema=True)

In [158]:
# Step 3: Analyze the data
data.printSchema()
data.show(5)
print("Total number of rows:", data.count())

for col in ["ocean_proximity"]:
    print(f"Unique values in {col}:")
    data.select(col).distinct().show()

for col in data.columns:
    print(f"NA rate in {col}: {data.filter(data[col].isNull()).count() / data.count()}")

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR B

In [159]:
# Do one-hot encoding for categorical columns
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
data = indexer.fit(data).transform(data)
encoder = OneHotEncoder(inputCol="ocean_proximity_index", outputCol="ocean_proximity_encoded")
encModel = encoder.fit(data)
data = encModel.transform(data)


data.show(5)
print("Total number of rows:", data.count())

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------------+-----------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_index|ocean_proximity_encoded|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------------+-----------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                  3.0|          (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                  3.0|          (4,[3],[1.0])|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|   

In [160]:
# Handle missing values in total_bedrooms column
data = data.fillna(data.approxQuantile("total_bedrooms", [0.5], 0.001)[0], subset=["total_bedrooms"])

In [161]:
def rfe_feature_selection(data, features, target_col, num_features, estimator):
    # Initialize remaining features and selected features
    remaining_features = features
    selected_features = []

    while len(selected_features) < num_features:
        # Create VectorAssembler for remaining features
        assembler = VectorAssembler(inputCols=remaining_features, outputCol="features")
        data_assembled = assembler.transform(data)

        # Train Random Forest model
        rf = estimator(labelCol=target_col, featuresCol="features")
        model = rf.fit(data_assembled)

        # Get feature importances
        importances = model.featureImportances

        # Convert SparseVector to dense representation
        importances_dense = importances.toArray()


        # Find the least important feature
        least_important_feature_index = np.argmin(importances_dense)

        least_important_feature = remaining_features[least_important_feature_index]


        # Remove least important feature from remaining features
        remaining_features.remove(least_important_feature)

        # Add least important feature to selected features
        selected_features.append(least_important_feature)

    # Select final features
    return data.select(selected_features + [target_col])


In [162]:
# Select reamining features except target column
features = [col for col in data.columns if col != "ocean_proximity_encoded" and col != "ocean_proximity"]

# Eleminate not necassary features
num_features = 8  # Smaller Values giving very bad results
data_filtered = rfe_feature_selection(data, features, "median_house_value", num_features, RandomForestRegressor)

In [163]:
print(data_filtered.columns)

['total_bedrooms', 'households', 'population', 'total_rooms', 'housing_median_age', 'longitude', 'latitude', 'median_income', 'median_house_value']


In [164]:
# Select reamining features except target column
features = [col for col in data_filtered.columns if col != "median_house_value"]
if "ocean_proximity_index" in features:
    features.remove("ocean_proximity_index")
    features.append("ocean_proximity_encoded")

# Create a VectorAssembler to combine features into a single vector
assembler = VectorAssembler(inputCols=features, outputCol="features")
data = assembler.transform(data)

# Scale features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
data = scaler.fit(data).transform(data)

# Split data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [165]:
# Create a Regression models
models = [
    ("Linear Regression", LinearRegression(labelCol="median_house_value", featuresCol="scaledFeatures")),
    ("Random Forest", RandomForestRegressor(labelCol="median_house_value", featuresCol="scaledFeatures")),
    ("Gradient-Boosted Tree", GBTRegressor(labelCol="median_house_value", featuresCol="scaledFeatures")),
    ("Decision Tree", DecisionTreeRegressor(labelCol="median_house_value", featuresCol="scaledFeatures")),
    ("Generalized Linear Regression", GeneralizedLinearRegression(labelCol="median_house_value", featuresCol="scaledFeatures")),
    ("Factorization Machines", FMRegressor(labelCol="median_house_value", featuresCol="scaledFeatures"))
]


In [166]:
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction")

for name, model in models:
    model = model.fit(train_data)
    predictions = model.transform(test_data)
    mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
    rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
    r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
    print(f"Model: {name}")
    print(f"    MSE: {mse}, RMSE: {rmse}, R-squared: {r2}")

Model: Linear Regression
    MSE: 5153988451.783638, RMSE: 71791.28395413776, R-squared: 0.6274906250019834
Model: Random Forest
    MSE: 5390224545.135438, RMSE: 73418.14860874278, R-squared: 0.6104164386102778
Model: Gradient-Boosted Tree
    MSE: 3210917002.299314, RMSE: 56664.953915972736, R-squared: 0.7679279461165479
Model: Decision Tree
    MSE: 5554288259.869394, RMSE: 74527.09748721866, R-squared: 0.5985585789338075
Model: Generalized Linear Regression
    MSE: 5153988451.783638, RMSE: 71791.28395413776, R-squared: 0.6274906250019834
Model: Factorization Machines
    MSE: 6482084990.653973, RMSE: 80511.39665074735, R-squared: 0.5315011954058797


In [172]:
# Train the best model
best_model = GBTRegressor(labelCol="median_house_value", featuresCol="scaledFeatures")
best_model = best_model.fit(train_data)


In [171]:
# Evaluate best model
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction")
predictions = best_model.transform(test_data)
mse = evaluator.evaluate(predictions, {evaluator.metricName: "mse"})
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
explained_variance = evaluator.evaluate(predictions, {evaluator.metricName: "var"})


# Print results
print(f"Best Model: Gradient-Boosted Tree")
print(f"    MSE: {mse}, RMSE: {rmse}, MAE: {mae}, R-squared: {r2}, Explained Variance: {explained_variance}")

Best Model: Gradient-Boosted Tree
    MSE: 3210917002.299314, RMSE: 56664.953915972736, MAE: 39837.32920705323, R-squared: 0.7679279461165479, Explained Variance: 10334394537.062147


In [175]:
# Train the best model on the entire dataset
best_model = GBTRegressor(labelCol="median_house_value", featuresCol="scaledFeatures")
best_model = best_model.fit(data)

# Save the model
best_model.save("best_model")

In [177]:
# Zip the model
import zipfile
with zipfile.ZipFile("best_model.zip", "w") as zf:
    zf.write("best_model/data/")
    zf.write("best_model/metadata/")
    zf.write("best_model/treesMetadata/")
